We're going to use a custom Cython implementation of the median filter.

In [1]:
import median_filter

In [2]:
import numpy as np
import pandas as pd
from skimage import measure


l = np.arange(480) - 1
l[0] = 0
r = np.arange(480) + 1
r[-1] = r[-2]
b = np.arange(640) - 1
b[0] = 0
u = np.arange(640) + 1
u[-1] = u[-2]


def expand(mask):
    new = mask.copy()
    
    for shift in [l, r]:
        new |= mask[shift, :]
    
    for shift in [b, u]:
        new |= mask[:, shift]

    return new


def find_interesting_pixels(img):
    
    med = median_filter.median_filter(img, 20)
    mask = img > med + 7
    
    labels = measure.label(expand(mask))
    
    return pd.DataFrame(
        [
            [*region.centroid, region.area, region.eccentricity, region.solidity]
            for region in measure.regionprops(labels)
        ],
        columns=['r', 'c', 'area', 'eccentricity', 'solidity']
    )

Example.

In [3]:
from PIL import Image

img = np.asarray(Image.open('data/spotGEO/train/10/1.png')).copy()
find_interesting_pixels(img).shape

(415, 5)

Do it for each image.

In [4]:
import pathlib
from joblib import Parallel, delayed
import tqdm

def f(part, seq, frame):
    img = np.array(Image.open(frame))
    return find_interesting_pixels(img).assign(part=part, sequence=int(seq.name), frame=int(frame.stem))

interesting = Parallel(n_jobs=4)(
    delayed(f)(part, seq, frame)
    for part in ['train', 'test']
    for seq in tqdm.tqdm(list(pathlib.Path(f'data/spotGEO/{part}').glob('*')), position=0)
    for frame in seq.glob('*.png')
)

interesting = pd.concat(interesting)
interesting = interesting.set_index(['part', 'sequence', 'frame']).sort_index()
interesting.to_pickle('data/interesting.pkl')

100%|██████████| 5120/5120 [23:35<00:00,  3.62it/s]  


Average number of interesting regions per image.

In [5]:
interesting.groupby(['part', 'sequence', 'frame']).size().mean()

260.93625

Percentage of pixels this represents.

In [6]:
f'{len(interesting) / (640 * 480 * 31996):%}'

'0.084951%'

Load the provided annotations.

In [7]:
import json
import pandas as pd

sats = []

with open('data/spotGEO/train_anno.json') as f:
    for ann in json.load(f):
        for i, coords in enumerate(ann['object_coords']):
            sats.append({
                'sequence': ann['sequence_id'],
                'frame': ann['frame'],
                'satellite': i + 1,
                'r': int(coords[1] + .5),
                'c': int(coords[0] + .5),
            })
    
sats = pd.DataFrame(sats)
sats = sats.set_index(['sequence', 'frame', 'satellite'])
sats.head()

r    c
sequence frame satellite          
1        1     1          237  502
               2          222  490
               3          129  141
         2     1          214  530
               2          199  518

Now let's annotate each interesting region.

In [8]:
from scipy import optimize
from scipy.spatial import distance

def assign_labels(interesting, satellites):
    
    # Compute the distance between each satellite and each interesting location,
    # thus forming a bipartite graph
    distances = distance.cdist(satellites, interesting)
    
    # Guess which locations correspond to which satellites
    row_ind, col_ind = optimize.linear_sum_assignment(distances)

    # Each satellite is assigned, but some of them may be too distant to be likely
    likely = distances[row_ind, col_ind] < 3
    
    labels = np.full(len(interesting), False, dtype=bool)
    labels[col_ind[likely]] = True
    return labels

Example.

In [9]:
interesting

r           c  area  eccentricity  solidity
part  sequence frame                                                      
test  1        1        2.000000  334.000000     5      0.000000  1.000000
               1        7.432836  339.350746   134      0.983181  0.853503
               1       10.727273  264.642857   154      0.978166  0.865169
               1        6.000000  321.000000     5      0.000000  1.000000
               1       18.993590   40.365385   156      0.975910  0.901734
...                          ...         ...   ...           ...       ...
train 1280     5      463.416667  290.000000    12      0.881527  0.857143
               5      474.173913   56.043478    46      0.935946  0.867925
               5      470.230769  291.230769    13      0.729661  0.928571
               5      477.357143  293.142857    14      0.678394  0.933333
               5      478.285714  107.000000     7      0.377964  1.000000

[8349960 rows x 5 columns]

In [10]:
assign_labels(interesting.loc['train', 1, 1][['r', 'c']], sats.loc[1, 1][['r', 'c']])

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False,  True, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False,  True, False, False, False, False, False,
       False, False, False,  True, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,

Now assign labels for each frame.

In [11]:
labels = pd.Series(dtype=bool, index=interesting.loc['train'].index)

for (sequence, frame), locations in tqdm.tqdm(interesting.loc['train'].groupby(['sequence', 'frame']), position=0):
    try:
        satellites = sats.loc[sequence, frame]
    except KeyError:
        continue
    labels.loc[sequence, frame] = assign_labels(locations[['r', 'c']], satellites[['r', 'c']])
    
interesting['is_satellite'] = None
interesting.loc['train', 'is_satellite'] = labels.values
interesting.to_pickle('data/interesting.pkl')

100%|██████████| 6400/6400 [00:39<00:00, 161.94it/s]


Determine the amount of satellites that got assigned.

In [12]:
interesting.loc['train']['is_satellite'].sum() / len(sats)

0.7941990182954038

In [13]:
0.6863900044622936

0.6863900044622936

Next, head to [Solution.ipynb](Solution.ipynb).